In [ ]:
import os
import json
import yaml

import numpy as np
import pandas as pd

from decode_trf import delivery_data_from_logfile

from pymedphys.level1.msqconnect import mosaiq_connect
from pymedphys.level1.configutilities import get_index, get_data_directory, get_filepath, get_gantry_tolerance
from pymedphys.level1.mudensity import find_relevant_control_points, calc_mu_density
from pymedphys.level2.msqdelivery import multi_fetch_and_verify_mosaiq
from pymedphys.level3.analyselogfiles import plot_results, calc_comparison
from pymedphys.level4.comparebygantry import (
    get_mappings, get_comparisons_byfield, get_logfile_delivery_data_bygantry,
    get_logfile_mu_density_bygantry, get_mosaiq_delivery_data_bygantry,
    get_mosaiq_mu_density_bygantry, get_comparison_results
)

In [ ]:
with open('../config.json') as config_file:
    config = json.load(config_file)

In [ ]:
index = get_index(config)

In [ ]:
data_directory = get_data_directory(config)

cache_filepath = os.path.join(data_directory, 'cache', 'dmlc_comparison.yaml')
cache_scratch_filepath = os.path.join(data_directory, 'cache', 'dmlc_comparison_scratch.yaml')

with open(cache_filepath, 'r') as cache_file:
    cache = yaml.load(cache_file)

In [ ]:
patient_ids = list(cache.keys())
len(patient_ids)

In [ ]:
index = get_index(config)
file_hashes = np.array(list(index.keys()))

field_types = np.array([
    index[file_hash]['delivery_details']['field_type']
    for file_hash in file_hashes
])

file_hashes = file_hashes[field_types == 'DMLC']

is_qa = np.array([
    index[file_hash]['delivery_details']['qa_mode']
    for file_hash in file_hashes
])

file_hashes = file_hashes[np.invert(is_qa)]

machine = np.array([
    index[file_hash]['logfile_header']['machine']
    for file_hash in file_hashes
])

# limit to RCCC for now
file_hashes = file_hashes[(machine == '2619') | (machine == '2694')]
np.random.shuffle(file_hashes)

# might need course information
index[file_hashes[0]]

In [ ]:
patient_grouped_fields, field_id_grouped_hashes = get_mappings(index, file_hashes)

In [ ]:
data = []

for patient_id in patient_ids:
    for byfield in cache[patient_id]:
        for angle in cache[patient_id][byfield]:
            comparison = cache[patient_id][byfield][angle]['median']
            file_hashes = cache[patient_id][byfield][angle]['median_filehash_group']
            data.append([
                patient_id, byfield, angle, comparison, file_hashes
            ])
            
comparisons_table = pd.DataFrame(
    columns=[
        'patient_id', 'field_id',
        'gantry_angle', 'comparison',
        'file_hashes'
    ],
    data=data
)

comparisons_table = comparisons_table.sort_values('comparison', ascending=False)
top_ten = comparisons_table.iloc[0:10]
top_ten

In [ ]:
len(comparisons_table)

In [ ]:
# field_id = int(comparisons_table.iloc[0]['field_id'])
# field_id

In [ ]:
logfile_group = comparisons_table.iloc[0]['file_hashes']
logfile_group

In [ ]:
gantry_angle = comparisons_table.iloc[0]['gantry_angle']
gantry_angle

In [ ]:
field_ids = []

for logfile_hash in logfile_group:
    field_ids.append(index[logfile_hash]['delivery_details']['field_id'])

assert len(np.unique(field_ids)) == 1

field_id = field_ids[0]
field_id

In [ ]:
with mosaiq_connect('msqsql') as cursor:
    mosaiq_delivery_data = multi_fetch_and_verify_mosaiq(
        cursor, field_id)

In [ ]:
def extract_angle_from_delivery_data(delivery_data, gantry_angle, gantry_tolerance=0):
    moniter_units = np.array(delivery_data.monitor_units)
    relevant_control_points = find_relevant_control_points(moniter_units)
    
    mu = moniter_units[relevant_control_points]
    mlc = np.array(delivery_data.mlc)[relevant_control_points]
    jaw = np.array(delivery_data.jaw)[relevant_control_points]
    gantry_angles = np.array(delivery_data.gantry)[relevant_control_points]
    
    gantry_angle_within_tolerance = (
        np.abs(gantry_angles - gantry_angle) <= gantry_tolerance)
    diff_mu = np.concatenate([[0], np.diff(mu)])[gantry_angle_within_tolerance]
    mu = np.cumsum(diff_mu)
    
    mlc = mlc[gantry_angle_within_tolerance]
    jaw = jaw[gantry_angle_within_tolerance]
    
    return mu, mlc, jaw

In [ ]:
mosaiq_mu_density = calc_mu_density(
    *extract_angle_from_delivery_data(mosaiq_delivery_data, gantry_angle)
)

In [ ]:
logfile_mu_density = None

for filehash in logfile_group:
    filepath = get_filepath(index, config, filehash)
    logfile_delivery_data = delivery_data_from_logfile(filepath)
    
    gantry_tolerance = get_gantry_tolerance(index, filehash, config)
    
    a_logfile_mu_density = calc_mu_density(
        *extract_angle_from_delivery_data(logfile_delivery_data, gantry_angle, gantry_tolerance)
    )
    
    if logfile_mu_density is None:
        logfile_mu_density = a_logfile_mu_density
    else:
        assert np.all(logfile_mu_density[0] == a_logfile_mu_density[0])
        assert np.all(logfile_mu_density[1] == a_logfile_mu_density[1])
        logfile_mu_density[2] += a_logfile_mu_density[2]

In [ ]:
grid_xx = logfile_mu_density[0]
grid_yy = logfile_mu_density[1]

comparison = calc_comparison(logfile_mu_density[2], mosaiq_mu_density[2])

print(comparison)
plot_results(grid_xx, grid_yy, logfile_mu_density[2], mosaiq_mu_density[2])